In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Rovibrational matrix elements of Cartesian tensor operators

Compute matrix elements of spin-rotation tensors on $\text{H}_1$ and $\text{H}_2$ atoms of $\text{H}_2\text{S}$ as well as electric dipole-moment operator in the basis of rovibrational states.

In [4]:
from typing import Dict, List

import h5py
import numpy as np
from jax import config
from jax import numpy as jnp

from rovib.cartens import CART_IND
from rovib.spinrot_xy2 import spinrot_xy2
from rovib.symtop import threej_wang

config.update("jax_enable_x64", True)

The spin-rotation matrix elements are computed using the following formula

$$
M_{\omega,n}^{(J',l',J,l)}=\sum_{v',k'}\sum_{v,k}c_{v',k'}^{(J',\Gamma',l')*}c_{v,k}^{(J,\Gamma,l)}(-1)^{k'}\sum_{\sigma=-\omega}^{\omega}\sum_{\alpha,\beta=x,y,z}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha\beta}^{(\Omega)}\langle v'|\bar{M}_{\alpha\beta,n}|v\rangle.
$$

Here, $\Omega=2$ is the rank of spin-rotation tensor, $\omega=0..\Omega$, $n=1$ and 2 for $\text{H}_1$ and $\text{H}_2$, respectively,
$\bar{M}_{\alpha\beta,n}$ is the electronic spin-rotation tensor (see `h2s_spinrot.py` for details), $v$ and $k$ denote the vibrational and rotational quanta,
$J$ and $\Gamma$ denote 'good' quantum numbers of the rotational angular momentum and the state symmetry, and $l$ is the state running index within
a set of states with the same $J$ and $\Gamma$.
The rovibrational state expansion coefficients $c_{v,k}^{(J,\Gamma,l)}$
are computed in `h2s_rovib.ipynb` and stored in seperate files
for different values of $J$ quanta.
The matrix $U_{\omega\sigma,\alpha\beta}^{(\Omega)}$ transforms Cartesian tensor of rank $\Omega$ into its spherical-tensor representation
(see `cartens.UMAT_CART_TO_SPHER`).

The rotational part of the expression
$$
R_{\omega,k',k,\alpha\beta}^{(J',J)} = \sum_{\sigma=-\omega}^{\omega}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha\beta}^{(\Omega)}
$$
is computed by the `symtop.threej_wang` function where it is transformed from the symmetric-top basis $|J,k\rangle$ ($k=-J..J$) into the so-called Wang's symmetry-adapted basis $|J,k,\tau\rangle$ ($k=0..J$, $\tau=0,1$ is partity as $(-1)^\tau$).

The vibrational matrix elements
$$
V_{n,v',v,\alpha\beta} = \langle v'|\bar{M}_{\alpha\beta,n}|v\rangle
$$
are computed in `h2s_rovib.ipynb` and stored in file.

Define functions to load the energies and quantum numbers for selected rovibrational states (`rovib_states`) and to compute the above matrix elements (`tensor_rovib_me`).

In [5]:
def rovib_states(j: int, state_ind_list: Dict[str, List[int]] = None, pmax: int = 20):
    """Reads rovibrational energies and quanta stored in files for different values
    of the J quantum number (the files are created in `h2s_rovib.ipynb`).

    Optionally, `state_ind_list[symmetry][<list of state indices>]`
    can be used to select specific states by their indices
    for different symmetries.
    """
    h5 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j}.h5", "r")
    energies = {}
    quanta = {}
    for sym in h5["energies"].keys():
        enr = h5["energies"][sym][:]
        coefs = h5["coefficients"][sym][:]
        vind = h5["vib-indices"][sym][:]
        rind = h5["rot-indices"][sym][:]
        qua = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5["quanta"][sym][:]]
        )
        if state_ind_list is not None and sym in state_ind_list:
            energies[sym] = enr[state_ind_list[sym]]
            quanta[sym] = qua[state_ind_list[sym]]
        else:
            energies[sym] = enr
            quanta[sym] = qua
    return energies, quanta

In [6]:
def tensor_rovib_me(
    rank: int,
    j1: int,
    j2: int,
    vib_me: np.ndarray,
    state_ind_list1: Dict[str, List[int]] = None,
    state_ind_list2: Dict[str, List[int]] = None,
    pmax: int = 20,
    linear: bool = False,
    tol: float = 1e-12,
):
    """Computes rovibrational matrix elements of a Cartesian tensor operator.

    This function calculates the rovibrational matrix elements using wavefunctions stored in
    separate files for different values of the J quantum number (the files are creaed in `h2s_rovib.ipynb`).
    The computation is performed for the specified bra (`j1`) and ket (`j2`) values of the J
    quantum number.

    Optionally, `state_ind_list1[symmetry][<list of state indices>]` and `state_ind_list2[...]`
    can be used to select specific bra and ket states by their indices for different symmetries.
    """

    # determine the order of Cartesian indices in the Cartesian-to-spherical tensor
    #   transformation matrix (in cartens.CART_IND and symtop.threej_wang)
    cart_ind = [["xyz".index(x) for x in elem] for elem in CART_IND[rank]]

    # reshape vibrational matrix elements such that the order of Cartesian indices
    #   correspond to the order in symtop.threej_wang output
    vib_me2 = jnp.moveaxis(
        jnp.array([vib_me[:, :, i, j] for (i, j) in cart_ind]), 0, -1
    )

    # compute rotational matrix elements of three-j symbol contracted with
    #   Cartesian-to-spherical tensor transformation matrix
    jktau_list1, jktau_list2, rot_me = threej_wang(rank, j1, j2, linear=linear)
    # rot_me[omega].shape = (2*j1+1, 2*j2+1, ncart)

    h5_1 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j1}.h5", "r")
    h5_2 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j2}.h5", "r")

    res = {}

    for sym1 in h5_1["energies"].keys():
        enr1 = h5_1["energies"][sym1][:]
        coefs1 = h5_1["coefficients"][sym1][:]
        vind1 = h5_1["vib-indices"][sym1][:]
        rind1 = h5_1["rot-indices"][sym1][:]
        qua1 = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5_1["quanta"][sym1][:]]
        )

        if state_ind_list1 is not None:
            if sym1 in state_ind_list1:
                ind = state_ind_list1[sym1]
                enr1 = enr1[ind]
                coefs1 = coefs1[:, ind]
            else:
                continue

        for sym2 in h5_2["energies"].keys():
            enr2 = h5_2["energies"][sym2][:]
            coefs2 = h5_2["coefficients"][sym2][:]
            vind2 = h5_2["vib-indices"][sym2][:]
            rind2 = h5_2["rot-indices"][sym2][:]
            qua2 = np.array(
                [elem[0].decode("utf-8").split(",") for elem in h5_2["quanta"][sym2][:]]
            )

            if state_ind_list2 is not None:
                if sym2 in state_ind_list2:
                    ind = state_ind_list2[sym2]
                    enr2 = enr2[ind]
                    coefs2 = coefs2[:, ind]
                else:
                    continue

            vib_me_ = vib_me2[jnp.ix_(vind1, vind2)]

            me = []
            for omega in range(rank + 1):
                me_ = jnp.einsum(
                    "ijc,ijc->ij", vib_me_, rot_me[omega][jnp.ix_(rind1, rind2)]
                )
                me.append(jnp.einsum("ik,ij,jl->kl", jnp.conj(coefs1), me_, coefs2))

            me = jnp.moveaxis(jnp.array(me), 0, -1)
            me = jnp.where(jnp.abs(me) < tol, 0.0, me)
            if jnp.count_nonzero(me) > 0:
                res[(sym1, sym2)] = jnp.array(me)

    return res

Using the functions above, compute matrix elements of spin-rotation tensors and store them in files.

In [7]:
pmax = 20
rank = 2

min_J = 0
max_J = 4

# read vibrational matrix elements of spin-rotation tensors

with h5py.File(f"h2s_vibme_pmax{pmax}.h5", "r") as h5:
    sr_h1_vib = h5["spin-rotation"]["h1"][:]
    sr_h2_vib = h5["spin-rotation"]["h2"][:]

# compute rovibrational matrix elements and store them in file

for J1 in range(min_J, max_J + 1):
    for J2 in range(min_J, max_J + 1):
        sr1_me = tensor_rovib_me(rank, J1, J2, sr_h1_vib)
        sr2_me = tensor_rovib_me(rank, J1, J2, sr_h2_vib)
        if sr1_me or sr2_me:
            print(
                f"store matrix elements for J1 = {J1} J2 = {J2}, delta J = {abs(J1-J2)}"
            )
            h5 = h5py.File(f"h2s_spinrot_pmax{pmax}_j{J1}_j{J2}.h5", "w")
            h5_sr1 = h5.create_group("h1")
            h5_sr2 = h5.create_group("h2")
            for sym1, sym2 in sr1_me:
                h5_sr1.create_dataset(f"{sym1}/{sym2}", data=sr1_me[(sym1, sym2)])
                h5_sr2.create_dataset(f"{sym1}/{sym2}", data=sr2_me[(sym1, sym2)])

store matrix elements for J1 = 0 J2 = 0
store matrix elements for J1 = 0 J2 = 1
store matrix elements for J1 = 0 J2 = 2
store matrix elements for J1 = 1 J2 = 0
store matrix elements for J1 = 1 J2 = 1
store matrix elements for J1 = 1 J2 = 2
store matrix elements for J1 = 1 J2 = 3
store matrix elements for J1 = 2 J2 = 0
store matrix elements for J1 = 2 J2 = 1
store matrix elements for J1 = 2 J2 = 2
store matrix elements for J1 = 2 J2 = 3
store matrix elements for J1 = 2 J2 = 4
store matrix elements for J1 = 3 J2 = 1
store matrix elements for J1 = 3 J2 = 2
store matrix elements for J1 = 3 J2 = 3
store matrix elements for J1 = 3 J2 = 4
store matrix elements for J1 = 4 J2 = 2
store matrix elements for J1 = 4 J2 = 3
store matrix elements for J1 = 4 J2 = 4
